<a href="https://colab.research.google.com/github/chunyu0329/yu/blob/main/EX05_03_Reply_Multi_type_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

In [36]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import(
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,StickerMessage,ImageMessage,VideoMessage,LocationMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'
import google.generativeai as genai
genai.configure(api_key="API_key")
model = genai.GenerativeModel("gemini-2.0-flash")
def ask_gemini(question):
  response=model.generate_content(question)
  return response.text

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        action=event.message.text
        if action=='sticker':
          #回覆貼圖
          reply=StickerMessage(package_id='446',sticker_id='1989')
        elif action=='image':
          #回覆圖片
          reply=ImageMessage(
            original_content_url='https://cdn.pixabay.com/photo/2024/03/03/21/11/peach-blossoms-8611337_1280.jpg',
            preview_image_url='https://cdn.pixabay.com/photo/2024/03/03/21/11/peach-blossoms-8611337_1280.jpg'
          )
        elif action=='video':
          reply=VideoMessage(
            original_content_url='https://pixabay.com/videos/flowers-spring-cherry-blossoms-268528/',
            preview_image_url='https://pixabay.com/videos/flowers-spring-cherry-blossoms-268528/'
          )
        elif action=='location':
            reply=LocationMessage(
                title='Taipei 101',
                address='110台北市信義區信義路五段7號',
                latitude=25.034199559170606,
                longtitude=121.56456035396677
            )
        else:
          response=ask_gemini(action)
          reply=TextMessage(text=response)
        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

ERROR:pyngrok.process.ngrok:t=2025-05-13T06:57:41+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-05-13T06:57:41+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.